In [2]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import _mysql_connector

In [6]:
df = pd.read_csv("../datos/attack_limpio.csv", index_col = 0)
df.columns

Index(['Unnamed: 0.1', 'case_number', 'year', 'type', 'country', 'area',
       'location', 'activity', 'name', 'age', 'time', 'species_', 'href',
       'injury', 'date', 'mes', 'fatal_', 'sex2', 'species_tipo', 'edad'],
      dtype='object')

In [ ]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        # nuestra clase va a recibir dos parámetros que son fijos a lo largo de toda la BBDD, el nombre de la BBDD y 
        # la contraseña con el servidor. 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

    # método para crear la BBDD 
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
            
    # método para crear tablas  e insertar datos en ellas   
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            

    # método para comprobar si la ciudad que estamos intentando meter ya existe en la BBDD. 
    # En caso de que ya exista no será necesario que la insertemos de nuevo
    def check_pais (self):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{self.contraseña}",
                                      host='127.0.0.1',
                                      database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        # query para extraer los valores únicos de ciudades de la tabla de localidades 
        query_existe_pais = f"""
                SELECT DISTINCT pais FROM ataques
                """
        mycursor.execute(query_existe_pais)
        pais = mycursor.fetchall()
        return pais
    

In [ ]:
# tabla ataques

for indice, fila in df.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_ataques = f"""
                INSERT INTO ataques ('year', 'type', 'country', 'area',
       'location', 'activity', 'name', 'age', 'time', 'species_', 'href',
       'injury', 'date', 'mes', 'fatal_', 'sex2', 'species_tipo', 'edad') 
                VALUES ("{fila["year"]}", "{fila["type"]}", "{fila["country"]}","{fila["area"]}", "{fila["location"]}", "{fila["activity"]}",
                "{fila["name"]}", "{fila["age"]}", "{fila["time"]}", "{fila["specie_"]}, "{fila["href"]}", "{fila["injury"]}",
                "{fila["date"]}", "{fila["mes"]}", "{fila["fatal"]}", "{fila["sex2"]}", "{fila["species_tipo"]}", "{fila["edad"]}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    pais = carga.check_pais()
    
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad. 
 
    if len(pais) == 0 or fila['pais'] not in pais[0]: 
        carga.crear_insertar_tabla( query_ataques)

    else:
        print(f"{fila['pais']} ya esta en nuestra BBDD")

In [ ]:
tabla_ataques = ''' CREATE TABLE IF NOT EXISTS `ataques` (
    `id_ataques` INT NOT NULL AUTO_INCREMENT, 
    `year` INT NOT NULL, 
    `type` VARCHAR(50) DEFAULT NULL, 
    `country` VARCHAR(50) DEFAULT NULL,  
    `age` FLOAT DEFAULT NULL,
    `species_` VARCHAR(50) DEFAULT NULL,
    `fecha_limpia` VARCHAR (10) DEFAULT NULL,
    `fatal` CHAR (1) DEFAULT NULL,
    `sex` CHAR (1) DEFAULT NULL,
    `activity` VARCHAR (250) DEFAULT NULL,
    PRIMARY KEY (`id_ataques`))
    
    ENGINE = InnoDB;
    '''

In [ ]:
# tabla mediciones

for indice, fila in df.iterrows():
    
    id_ciudad = carga.sacar_id_ciudad(fila['ciudad'])
    id_fecha = carga.sacar_id_fecha( fila["timepoint"], fila["fecha"])
    
    query_medidas = f"""
                INSERT INTO mediciones (cloudcover, lifted_index, prec_type, prec_amount, temp2m, rh2m, weather, wind_direction, wind_speed, seeing, transparency, idtiempo, idlocalidad) 
                VALUES ({fila["cloudcover"]}, {fila['lifted_index']}, "{fila['prec_type']}", {fila['prec_amount']},{fila['temp2m']},{fila['rh2m']}, "{fila['weather']}","{fila['wind10m.direction']}",{fila['wind10m.speed']}, {fila['seeing']}, {fila['transparency']}, {id_fecha}, {id_ciudad});
                """
    carga.crear_insertar_tabla( query_medidas)
    